In [1]:
import argparse
import itertools

In [29]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import hiddenlayer as hl
import graphviz
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [30]:
import glob
import random
import os
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import Dataset
from PIL import Image


In [31]:
import visdom
import numpy as np
# vis = visdom.Visdom()
# vis.text('Hello, world!')
# vis.image(np.ones((3, 10, 10)))

In [44]:
import random
import time
import datetime
import sys

from torch.autograd import Variable
import torch
from visdom import Visdom
import numpy as np        
        
        

        

In [45]:
#https://github.com/jxgu1016/Total_Variation_Loss.pytorch/blob/master/TVLoss.py
class TVLoss(nn.Module):
    def __init__(self,TVLoss_weight= 1):
        super(TVLoss,self).__init__()
        self.TVLoss_weight = TVLoss_weight

    def forward(self,x):
        batch_size = x.size()[0]
        h_x = x.size()[2]
        w_x = x.size()[3]
        count_h = self._tensor_size(x[:,:,1:,:])
        count_w = self._tensor_size(x[:,:,:,1:])
        h_tv = torch.pow((x[:,:,1:,:]-x[:,:,:h_x-1,:]),2).sum()
        w_tv = torch.pow((x[:,:,:,1:]-x[:,:,:,:w_x-1]),2).sum()
        return self.TVLoss_weight*2*(h_tv/count_h+w_tv/count_w)/batch_size

    def _tensor_size(self,t):
        return t.size()[1]*t.size()[2]*t.size()[3]
    


In [46]:
class Identity(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__()
    def forward(self, x):
        return x
    
    
class ResidualBlock(nn.Module):
    def __init__(self,in_features):
        super(ResidualBlock,self).__init__()
        conv_block = [  nn.ReflectionPad2d(1),
                        nn.Conv2d(in_features, in_features, 3),
                        nn.InstanceNorm2d(in_features),
                        nn.ReLU(inplace=True),
                        nn.ReflectionPad2d(1),
                        nn.Conv2d(in_features, in_features, 3),
                        nn.InstanceNorm2d(in_features)  ]

        self.conv_block = nn.Sequential(*conv_block)

    def forward(self, x):
        return x + self.conv_block(x) #skip connection    

class Encoder(nn.Module):    
    def __init__(self, in_nc, ngf=64):
        super(Encoder, self).__init__()
        
        #Inital Conv Block
        model = [   nn.ReflectionPad2d(3),
                    nn.Conv2d(in_nc, ngf, 7),
                    nn.InstanceNorm2d(ngf),
                    nn.ReLU(inplace=True) ]
        
        in_features = ngf
        out_features = in_features *2
        
        for _ in range(2):
            model += [
                nn.Conv2d(in_features, out_features, 3, stride=2, padding=1),
                nn.InstanceNorm2d(out_features),
                nn.ReLU(inplace=True)    
            ]
            
            in_features = out_features
            out_features = in_features * 2
            
        self.model = nn.Sequential(*model)
        
    def forward(self,x):
        #Return batch w/ encoded content picture
        return [self.model(x['content']),
               x['style_label']]
    
class Transformer(nn.Module):
    def __init__(self,n_styles, ngf,auto_id=True):
        super(Transformer, self).__init__()
        
        #nclasses = input_nclasses
        self.t = nn.ModuleList([ResidualBlock(ngf*4) for i in range(n_styles)])
        if auto_id:
            self.t.append(Identity())
        #self.i = Identity()
                
    def forward(self,x):
        #x0 is content, x1 is label 
        label = x[1][0]
#         print(label)
#         print(len(label))
#         print(label.shape)
        mix = sum([self.t[i](x[0])*v for (i,v) in enumerate(label)])
        #return content transformed by style specific residual block 
        return mix
        
class Decoder(nn.Module):
    def __init__(self, out_nc, ngf, n_residual_blocks=9):
        super(Decoder, self).__init__()
        
        in_features = ngf * 4
        out_features = in_features//2
        
        model = []
        #ResBlockLand
        
        for _ in range(n_residual_blocks):
            model += [ResidualBlock(in_features)]

        # Upsampling
        for _ in range(2):
            model += [  nn.ConvTranspose2d(in_features, out_features, 3, stride=2, padding=1, output_padding=1),
                        nn.InstanceNorm2d(out_features),
                        nn.ReLU(inplace=True) ]
            in_features = out_features
            out_features = in_features//2

        # Output layer
        model += [  nn.ReflectionPad2d(3),
                    nn.Conv2d(64, out_nc, 7),
                    nn.Tanh() ]
        
        self.model = nn.Sequential(*model)
        
    def forward(self,x):
        return self.model(x)
        
class Generator(nn.Module):
    def __init__(self,in_nc,out_nc,n_styles,ngf):
        super(Generator, self).__init__()
        
        self.encoder = Encoder(in_nc,ngf)
        self.transformer = Transformer(n_styles,ngf)
        self.decoder = Decoder(out_nc,ngf)
        
    def forward(self,x):
        #Pass generator batch of {content=,style=?,style_label=}
        print('generator shape/style in: ',x['content'].shape, x['style_label'])
        
        e = self.encoder(x)
        print(e[0].shape,e[1])
        t = self.transformer(e)
        print(t.shape)
        d = self.decoder(t)
        print(d.shape)
        return d
    
class Discriminator(nn.Module):
    def __init__(self, input_nc, n_styles, ndf=64):
        super(Discriminator, self).__init__()

        # A bunch of convolutions one after another
        model = [   nn.Conv2d(in_nc, 64, 4, stride=2, padding=1),
                    nn.LeakyReLU(0.2, inplace=True) ]

        model += [  nn.Conv2d(64, 128, 4, stride=2, padding=1),
                    nn.InstanceNorm2d(128), 
                    nn.LeakyReLU(0.2, inplace=True) ]

        model += [  nn.Conv2d(128, 256, 4, stride=2, padding=1),
                    nn.InstanceNorm2d(256), 
                    nn.LeakyReLU(0.2, inplace=True) ]

        model += [  nn.Conv2d(256, 512, 4, padding=1),
                    nn.InstanceNorm2d(512), 
                    nn.LeakyReLU(0.2, inplace=True) ]
        
        
        self.model = nn.Sequential(*model)

        # FCN classification layer-
        self.fldiscriminator = nn.Conv2d(512, 1, 4, padding=1)
        
        # aux class layer
        self.aux_clf = nn.Conv2d(512, n_styles, 4, padding = 4)

    def forward(self, x):
        base =  self.model(x)
        print('base shape: ',base.shape)
        discrim = self.fldiscriminator(base)
        print('init discrim shape: ',discrim.shape)
        # Average pooling and flatten
        discrim=F.avg_pool2d(discrim, discrim.size()[2:]).view(discrim.size()[0], -1) 
        print('preview discrim: ',discrim)
        print('preview discrim shape: ',discrim.size())
        discrim = discrim.view(-1)
        print('discrim shape: ',discrim.size())
        print('discrim: ',discrim)
        clf = self.aux_clf(base).transpose_(1,3)
        print('clf shape: ',clf.shape)
        print('clf transpose: ',clf.transpose_(1,3).shape)
        
        return [discrim,clf.transpose_(1,3)]
            

In [47]:
#TRAIN OPTIONS FROM GATED GAN
epoch = 0
n_epochs = 200 #default = 200
batchSize = 1
dataroot = './photo2fourcollection'
loadSize = 143
fineSize = 128
ngf = 64
ndf = 64    
in_nc = 3 
out_nc = 3 
niter = 100  
niter_decay = 100 
lr = 0.0002 
beta1 = 0.5 
#ntrain = math.huge 
flip = 1  
display_id = 10 
display_winsize = 128 
display_freq = 25 
gpu = 1 
name = ''   
which_direction = 'AtoB'
phase = 'train'
nThreads = 2
save_epoch_freq = 1
save_latest_freq = 5000 
print_freq = 50
save_display_feq = 2500
continue_train = 0
serial_batches = 0
checkpoints_dir = './checkpoints'
cudnn = 1
which_model_netD = 'basic'
which_model_netG = 'auto_gated_resnet_6blocks'
norm = 'instance'
n_layers_D = 3
lambda_A = 10.0
lambda_B = 10.0
model = 'gated_gan'
use_lsgan = 1
align_data = 0
pool_size = 50
resize_or_crop = 'resize_and_crop'
autoencoder_constrain = 10 
n_styles = 4
test_data_path = ''
decay_epoch=1
cuda=False
tv_strength=1e-6


In [57]:
dset=ImageDataset('./photo2fourcollection/')

In [60]:
len(dset.X)

6287

In [56]:
dataloader = DataLoader(ImageDataset('./photo2fourcollection'), 
                        batch_size=1, shuffle=True)
batch = next(iter(dataloader))


In [49]:
generator = Generator(in_nc, out_nc, n_styles, ngf)
discriminator= Discriminator(in_nc,n_styles, ndf)

In [50]:
#Losses Init
use_lsgan=True
if use_lsgan:
    criterion_GAN = nn.MSELoss()
else: 
    criterion_GAN = nn.BCELoss()
    
    
criterion_ACGAN = nn.CrossEntropyLoss(weight=None)
criterion_Rec = nn.L1Loss()
criterion_Enc = nn.MSELoss()
criterion_TV = TVLoss(TVLoss_weight=tv_strength)




In [51]:
#Optimizers & LR schedulers
optimizer_G = torch.optim.Adam(generator.parameters(),
                                lr=lr, betas=(0.5, 0.999))
optimizer_D = torch.optim.Adam(discriminator.parameters(), 
                               lr=lr, betas=(0.5, 0.999))


lr_scheduler_G = torch.optim.lr_scheduler.LambdaLR(optimizer_G, lr_lambda=LambdaLR(n_epochs, epoch,decay_epoch).step)
lr_scheduler_D = torch.optim.lr_scheduler.LambdaLR(optimizer_D, lr_lambda=LambdaLR(n_epochs,epoch, decay_epoch).step)



In [52]:
#Set vars for training
Tensor = torch.cuda.FloatTensor if cuda else torch.Tensor
input_A = Tensor(batchSize, in_nc, fineSize, fineSize)
input_B = Tensor(batchSize, out_nc, fineSize, fineSize)
target_real = Variable(Tensor(batchSize).fill_(1.0), requires_grad=False)
target_fake = Variable(Tensor(batchSize).fill_(0.0), requires_grad=False)

D_A_size = discriminator(batch['style'])[0].size()  
D_AC_size = discriminator(batch['style'])[1].size()

class_label_B = torch.Tensor(D_AC_size[0],D_AC_size[1],D_AC_size[2]).long()

autoflag_OHE = torch.Tensor(1,n_styles+1).fill_(0).long()
autoflag_OHE[0][-1] = 1

fake_label = torch.Tensor(D_A_size).fill_(0.0)
real_label = torch.Tensor(D_A_size).fill_(0.9) 

rec_A_AE = torch.Tensor(batchSize,in_nc,fineSize,fineSize)

fake_buffer = ReplayBuffer()

##INIT THOSE WEIGHTS INIT
generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)

base shape:  torch.Size([1, 512, 15, 15])
init discrim shape:  torch.Size([1, 1, 14, 14])
preview discrim:  tensor([[-0.3758]], grad_fn=<ViewBackward>)
preview discrim shape:  torch.Size([1, 1])
discrim shape:  torch.Size([1])
discrim:  tensor([-0.3758], grad_fn=<ViewBackward>)
clf shape:  torch.Size([1, 20, 20, 4])
clf transpose:  torch.Size([1, 4, 20, 20])
base shape:  torch.Size([1, 512, 15, 15])
init discrim shape:  torch.Size([1, 1, 14, 14])
preview discrim:  tensor([[-0.3758]], grad_fn=<ViewBackward>)
preview discrim shape:  torch.Size([1, 1])
discrim shape:  torch.Size([1])
discrim:  tensor([-0.3758], grad_fn=<ViewBackward>)
clf shape:  torch.Size([1, 20, 20, 4])
clf transpose:  torch.Size([1, 4, 20, 20])


Discriminator(
  (model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (3): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (4): LeakyReLU(negative_slope=0.2, inplace)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (6): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (7): LeakyReLU(negative_slope=0.2, inplace)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
    (9): InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (10): LeakyReLU(negative_slope=0.2, inplace)
  )
  (fldiscriminator): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
  (aux_clf): Conv2d(512, 4, kernel_size=(4, 4), stride=(1, 1), padding=(4, 4))
)

In [53]:
# real_content = Variable(input_A.copy_(batch['content']))
# real_style = Variable(input_B.copy_(batch['style']))
# style_label = batch['style_label']
# style_OHE = F.one_hot(style_label,n_styles).long()


In [54]:
logger = Logger(n_epochs, len(dataloader))


In [55]:
###FUCKIN TRAIN TIME BABY
for epoch in range(epoch,n_epochs):
    for i, batch in enumerate(dataloader):
        real_content = Variable(input_A.copy_(batch['content']))
        real_style = Variable(input_B.copy_(batch['style']))
        style_label = batch['style_label']
        style_OHE = F.one_hot(style_label,n_styles).long()
        
        class_label = class_label_B.copy_(label2tensor(style_label,class_label_B)).long()
        
        #### GENERATOR FORWARD
        optimizer_G.zero_grad()
        #Auto Encoder Reconstruction Loss
        rec = generator({
            'content':real_content,
            'style_label': autoflag_OHE # 1-n_styles +1 ((nstyles))
        })
        errRec = criterion_Rec(rec,real_content)

        #Gan Loss
        genfake = generator({
            'content':real_content,
            'style_label': style_OHE
        })
        output = discriminator(genfake)
        errG = criterion_GAN(output[0],torch.tensor([0.]))

        #https://github.com/pytorch/pytorch/issues/29
        #Aux Class Loss
        errG_AC = criterion_ACGAN(output[1].transpose_(1,3),class_label)

        tvloss = criterion_TV(genfake)
        errG_total = errRec*autoencoder_constrain + errG_AC*lambda_A + errG + tvloss

        errG_total.backward()
        optimizer_G.step()
        
        #### DISCRIMINATOR FORWARD
        optimizer_D.zero_grad()

        #Real Loss
        output = discriminator(real_style)
        errD_real = criterion_GAN(output[0],torch.tensor([.9]))

        errD_real_class = criterion_ACGAN(output[1].transpose(1,3),class_label_B)

        #Fake Loss
        fake = fake_buffer.push_and_pop(genfake)
        out_real, out_class = discriminator(fake)
        errD_fake = criterion_GAN(out_real,torch.tensor([0.]))

        errD = ((errD_real+errD_fake)/2.0)+errD_real_class
        errD.backward()

        optimizer_D.step()
        
        #Progress report (port 8097)
        logger.log({'loss_G': errG_total, 'loss_G_AE': errRec, 'loss_G_GAN': errG,
                    'loss_G_AC': errG_AC, 'loss_D': errD, 'errD_fake':errD_fake,
                   'errD_real':errD_real, 'errD_class': errD_real_class}, 
                    images={'content': real_content, 'style': real_style, 'transfer': genfake})
    
    ##update learning rates
    lr_scheduler_G.step()
    lr_scheduler_D_A.step()
    
    #Save model
    torch.save(netG.state_dict(), 'output/netG.pth')
    torch.save(netD.state_dict(), 'output/netD.pth')

        


generator shape/style in:  torch.Size([1, 3, 128, 128]) tensor([[0, 0, 0, 0, 1]])
torch.Size([1, 256, 32, 32]) tensor([[0, 0, 0, 0, 1]])
torch.Size([1, 256, 32, 32])
torch.Size([1, 3, 128, 128])
generator shape/style in:  torch.Size([1, 3, 128, 128]) tensor([[0, 0, 0, 1]])
torch.Size([1, 256, 32, 32]) tensor([[0, 0, 0, 1]])
torch.Size([1, 256, 32, 32])
torch.Size([1, 3, 128, 128])
base shape:  torch.Size([1, 512, 15, 15])
init discrim shape:  torch.Size([1, 1, 14, 14])
preview discrim:  tensor([[-0.3304]], grad_fn=<ViewBackward>)
preview discrim shape:  torch.Size([1, 1])
discrim shape:  torch.Size([1])
discrim:  tensor([-0.3304], grad_fn=<ViewBackward>)
clf shape:  torch.Size([1, 20, 20, 4])
clf transpose:  torch.Size([1, 4, 20, 20])
base shape:  torch.Size([1, 512, 15, 15])
init discrim shape:  torch.Size([1, 1, 14, 14])
preview discrim:  tensor([[-0.2494]], grad_fn=<ViewBackward>)
preview discrim shape:  torch.Size([1, 1])
discrim shape:  torch.Size([1])
discrim:  tensor([-0.2494], 

Epoch 005/010 [0001/6287] -- loss_G: 17.1567 | loss_G_AE: 0.5532 | loss_G_GAN: 0.1034 | loss_G_AC: 1.1522 | loss_D: 1.2360 | errD_fake: 0.1034 | errD_real: 0.0813 | errD_class: 1.1436 -- ETA: 0:00:29.335989
generator shape/style in:  torch.Size([1, 3, 128, 128]) tensor([[0, 0, 0, 0, 1]])
torch.Size([1, 256, 32, 32]) tensor([[0, 0, 0, 0, 1]])
torch.Size([1, 256, 32, 32])
torch.Size([1, 3, 128, 128])
generator shape/style in:  torch.Size([1, 3, 128, 128]) tensor([[1, 0, 0, 0]])
torch.Size([1, 256, 32, 32]) tensor([[1, 0, 0, 0]])
torch.Size([1, 256, 32, 32])
torch.Size([1, 3, 128, 128])
base shape:  torch.Size([1, 512, 15, 15])
init discrim shape:  torch.Size([1, 1, 14, 14])
preview discrim:  tensor([[0.1564]], grad_fn=<ViewBackward>)
preview discrim shape:  torch.Size([1, 1])
discrim shape:  torch.Size([1])
discrim:  tensor([0.1564], grad_fn=<ViewBackward>)
clf shape:  torch.Size([1, 20, 20, 4])
clf transpose:  torch.Size([1, 4, 20, 20])
base shape:  torch.Size([1, 512, 15, 15])
init dis

Epoch 010/010 [0001/6287] -- loss_G: 22.8420 | loss_G_AE: 0.3682 | loss_G_GAN: 0.0001 | loss_G_AC: 1.9160 | loss_D: 2.1353 | errD_fake: 0.0001 | errD_real: 0.4661 | errD_class: 1.9022 -- ETA: 0:00:03.301519
generator shape/style in:  torch.Size([1, 3, 128, 128]) tensor([[0, 0, 0, 0, 1]])
torch.Size([1, 256, 32, 32]) tensor([[0, 0, 0, 0, 1]])
torch.Size([1, 256, 32, 32])
torch.Size([1, 3, 128, 128])
generator shape/style in:  torch.Size([1, 3, 128, 128]) tensor([[0, 1, 0, 0]])
torch.Size([1, 256, 32, 32]) tensor([[0, 1, 0, 0]])
torch.Size([1, 256, 32, 32])
torch.Size([1, 3, 128, 128])
base shape:  torch.Size([1, 512, 15, 15])
init discrim shape:  torch.Size([1, 1, 14, 14])
preview discrim:  tensor([[0.8400]], grad_fn=<ViewBackward>)
preview discrim shape:  torch.Size([1, 1])
discrim shape:  torch.Size([1])
discrim:  tensor([0.8400], grad_fn=<ViewBackward>)
clf shape:  torch.Size([1, 20, 20, 4])
clf transpose:  torch.Size([1, 4, 20, 20])
base shape:  torch.Size([1, 512, 15, 15])
init dis

Epoch 015/010 [0001/6287] -- loss_G: 17.2409 | loss_G_AE: 0.4620 | loss_G_GAN: 0.8586 | loss_G_AC: 1.1762 | loss_D: 1.6796 | errD_fake: 0.8586 | errD_real: 0.1002 | errD_class: 1.2002 -- ETA: -1 day, 23:59:48.301692
generator shape/style in:  torch.Size([1, 3, 128, 128]) tensor([[0, 0, 0, 0, 1]])
torch.Size([1, 256, 32, 32]) tensor([[0, 0, 0, 0, 1]])
torch.Size([1, 256, 32, 32])
torch.Size([1, 3, 128, 128])
generator shape/style in:  torch.Size([1, 3, 128, 128]) tensor([[0, 1, 0, 0]])
torch.Size([1, 256, 32, 32]) tensor([[0, 1, 0, 0]])
torch.Size([1, 256, 32, 32])
torch.Size([1, 3, 128, 128])
base shape:  torch.Size([1, 512, 15, 15])
init discrim shape:  torch.Size([1, 1, 14, 14])
preview discrim:  tensor([[-0.0813]], grad_fn=<ViewBackward>)
preview discrim shape:  torch.Size([1, 1])
discrim shape:  torch.Size([1])
discrim:  tensor([-0.0813], grad_fn=<ViewBackward>)
clf shape:  torch.Size([1, 20, 20, 4])
clf transpose:  torch.Size([1, 4, 20, 20])
base shape:  torch.Size([1, 512, 15, 15

Epoch 020/010 [0001/6287] -- loss_G: 18.3416 | loss_G_AE: 0.3626 | loss_G_GAN: 0.2879 | loss_G_AC: 1.4427 | loss_D: 1.6684 | errD_fake: 0.2879 | errD_real: 0.1806 | errD_class: 1.4341 -- ETA: -1 day, 23:59:35.215822
generator shape/style in:  torch.Size([1, 3, 128, 128]) tensor([[0, 0, 0, 0, 1]])
torch.Size([1, 256, 32, 32]) tensor([[0, 0, 0, 0, 1]])
torch.Size([1, 256, 32, 32])
torch.Size([1, 3, 128, 128])
generator shape/style in:  torch.Size([1, 3, 128, 128]) tensor([[0, 0, 1, 0]])
torch.Size([1, 256, 32, 32]) tensor([[0, 0, 1, 0]])
torch.Size([1, 256, 32, 32])
torch.Size([1, 3, 128, 128])
base shape:  torch.Size([1, 512, 15, 15])
init discrim shape:  torch.Size([1, 1, 14, 14])
preview discrim:  tensor([[0.7165]], grad_fn=<ViewBackward>)
preview discrim shape:  torch.Size([1, 1])
discrim shape:  torch.Size([1])
discrim:  tensor([0.7165], grad_fn=<ViewBackward>)
clf shape:  torch.Size([1, 20, 20, 4])
clf transpose:  torch.Size([1, 4, 20, 20])
base shape:  torch.Size([1, 512, 15, 15])

...

Epoch 110/010 [0001/6287] -- loss_G: 17.4897 | loss_G_AE: 0.3112 | loss_G_GAN: 0.0129 | loss_G_AC: 1.4365 | loss_D: 1.5791 | errD_fake: 0.0936 | errD_real: 0.0005 | errD_class: 1.5320 -- ETA: -1 day, 23:56:05.534502
generator shape/style in:  torch.Size([1, 3, 128, 128]) tensor([[0, 0, 0, 0, 1]])
torch.Size([1, 256, 32, 32]) tensor([[0, 0, 0, 0, 1]])
torch.Size([1, 256, 32, 32])
torch.Size([1, 3, 128, 128])
generator shape/style in:  torch.Size([1, 3, 128, 128]) tensor([[0, 1, 0, 0]])
torch.Size([1, 256, 32, 32]) tensor([[0, 1, 0, 0]])
torch.Size([1, 256, 32, 32])
torch.Size([1, 3, 128, 128])
base shape:  torch.Size([1, 512, 15, 15])
init discrim shape:  torch.Size([1, 1, 14, 14])
preview discrim:  tensor([[0.0827]], grad_fn=<ViewBackward>)
preview discrim shape:  torch.Size([1, 1])
discrim shape:  torch.Size([1])
discrim:  tensor([0.0827], grad_fn=<ViewBackward>)
clf shape:  torch.Size([1, 20, 20, 4])
clf transpose:  torch.Size([1, 4, 20, 20])
base shape:  torch.Size([1, 512, 15, 15])

Epoch 115/010 [0001/6287] -- loss_G: 9.2463 | loss_G_AE: 0.2428 | loss_G_GAN: 0.0466 | loss_G_AC: 0.6772 | loss_D: 0.6741 | errD_fake: 0.0000 | errD_real: 0.0150 | errD_class: 0.6666 -- ETA: -1 day, 23:55:53.961273
generator shape/style in:  torch.Size([1, 3, 128, 128]) tensor([[0, 0, 0, 0, 1]])
torch.Size([1, 256, 32, 32]) tensor([[0, 0, 0, 0, 1]])
torch.Size([1, 256, 32, 32])
torch.Size([1, 3, 128, 128])
generator shape/style in:  torch.Size([1, 3, 128, 128]) tensor([[0, 1, 0, 0]])
torch.Size([1, 256, 32, 32]) tensor([[0, 1, 0, 0]])
torch.Size([1, 256, 32, 32])
torch.Size([1, 3, 128, 128])
base shape:  torch.Size([1, 512, 15, 15])
init discrim shape:  torch.Size([1, 1, 14, 14])
preview discrim:  tensor([[0.2985]], grad_fn=<ViewBackward>)
preview discrim shape:  torch.Size([1, 1])
discrim shape:  torch.Size([1])
discrim:  tensor([0.2985], grad_fn=<ViewBackward>)
clf shape:  torch.Size([1, 20, 20, 4])
clf transpose:  torch.Size([1, 4, 20, 20])
base shape:  torch.Size([1, 512, 15, 15])


Epoch 120/010 [0001/6287] -- loss_G: 9.6211 | loss_G_AE: 0.2501 | loss_G_GAN: 0.1716 | loss_G_AC: 0.6948 | loss_D: 0.8538 | errD_fake: 0.1716 | errD_real: 0.1730 | errD_class: 0.6814 -- ETA: -1 day, 23:55:42.437637
generator shape/style in:  torch.Size([1, 3, 128, 128]) tensor([[0, 0, 0, 0, 1]])
torch.Size([1, 256, 32, 32]) tensor([[0, 0, 0, 0, 1]])
torch.Size([1, 256, 32, 32])
torch.Size([1, 3, 128, 128])
generator shape/style in:  torch.Size([1, 3, 128, 128]) tensor([[1, 0, 0, 0]])
torch.Size([1, 256, 32, 32]) tensor([[1, 0, 0, 0]])
torch.Size([1, 256, 32, 32])
torch.Size([1, 3, 128, 128])
base shape:  torch.Size([1, 512, 15, 15])
init discrim shape:  torch.Size([1, 1, 14, 14])
preview discrim:  tensor([[0.0744]], grad_fn=<ViewBackward>)
preview discrim shape:  torch.Size([1, 1])
discrim shape:  torch.Size([1])
discrim:  tensor([0.0744], grad_fn=<ViewBackward>)
clf shape:  torch.Size([1, 20, 20, 4])
clf transpose:  torch.Size([1, 4, 20, 20])
base shape:  torch.Size([1, 512, 15, 15])


Epoch 125/010 [0001/6287] -- loss_G: 14.4045 | loss_G_AE: 0.3239 | loss_G_GAN: 0.0462 | loss_G_AC: 1.1119 | loss_D: 1.0894 | errD_fake: 0.0053 | errD_real: 0.0671 | errD_class: 1.0532 -- ETA: -1 day, 23:55:30.538362
generator shape/style in:  torch.Size([1, 3, 128, 128]) tensor([[0, 0, 0, 0, 1]])
torch.Size([1, 256, 32, 32]) tensor([[0, 0, 0, 0, 1]])
torch.Size([1, 256, 32, 32])
torch.Size([1, 3, 128, 128])
generator shape/style in:  torch.Size([1, 3, 128, 128]) tensor([[0, 0, 0, 1]])
torch.Size([1, 256, 32, 32]) tensor([[0, 0, 0, 1]])
torch.Size([1, 256, 32, 32])
torch.Size([1, 3, 128, 128])
base shape:  torch.Size([1, 512, 15, 15])
init discrim shape:  torch.Size([1, 1, 14, 14])
preview discrim:  tensor([[0.0099]], grad_fn=<ViewBackward>)
preview discrim shape:  torch.Size([1, 1])
discrim shape:  torch.Size([1])
discrim:  tensor([0.0099], grad_fn=<ViewBackward>)
clf shape:  torch.Size([1, 20, 20, 4])
clf transpose:  torch.Size([1, 4, 20, 20])
base shape:  torch.Size([1, 512, 15, 15])

KeyboardInterrupt: 